In [1]:
from IPython.display import display, HTML
display(HTML("<style>.container { width:95% !important; }</style>"))
import warnings
warnings.filterwarnings('ignore')

In [2]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import os
import statsmodels.formula.api as smf 
import statsmodels.api as sm
from patsy import dmatrices
from statsmodels.formula.api import ols
import statsmodels.formula.api as smf


Bad key text.latex.preview in file /Users/vorsi/opt/anaconda3/lib/python3.8/site-packages/matplotlib/mpl-data/stylelib/_classic_test.mplstyle, line 123 ('text.latex.preview : False')
You probably need to get an updated matplotlibrc file from
https://github.com/matplotlib/matplotlib/blob/v3.7.4/lib/matplotlib/mpl-data/matplotlibrc
or from the matplotlib source distribution

Bad key mathtext.fallback_to_cm in file /Users/vorsi/opt/anaconda3/lib/python3.8/site-packages/matplotlib/mpl-data/stylelib/_classic_test.mplstyle, line 155 ('mathtext.fallback_to_cm : True  # When True, use symbols from the Computer Modern')
You probably need to get an updated matplotlibrc file from
https://github.com/matplotlib/matplotlib/blob/v3.7.4/lib/matplotlib/mpl-data/matplotlibrc
or from the matplotlib source distribution

Bad key savefig.jpeg_quality in file /Users/vorsi/opt/anaconda3/lib/python3.8/site-packages/matplotlib/mpl-data/stylelib/_classic_test.mplstyle, line 418 ('savefig.jpeg_quality: 95       

In [3]:
from scipy import stats

In [4]:
def save_results(p_out, res, data, model):
    d={'aic':res.aic,
  "bic": res.bic,
  'r2':res.rsquared,
  "model":model,
      'data':data}
    t=res.summary2().tables[1]
    t.to_excel(p_out+model+"_"+str(data)+".xlsx")
    return t, pd.DataFrame.from_dict(d,'index').T

In [78]:
def save_results_logit(p_out, res, data, model):
    d={'aic':res.aic,
  "bic": res.bic,
  'r2':res.prsquared,
  "model":model,
      'data':data}
    t=res.summary2().tables[1]
    t.to_excel(p_out+model+"_"+str(data)+".xlsx")
    return t, pd.DataFrame.from_dict(d,'index').T

In [13]:
# GITHUB LINKS
p_in_gs="/Users/vorsi/Dropbox/behance-data/NEW_ANALYSIS/data/robustness_github/"
p_in_g05='/Users/vorsi/Dropbox/behance-data/NEW_ANALYSIS/data/robbustness_github_gender_swap/r05/'
p_in_g1='/Users/vorsi/Dropbox/behance-data/NEW_ANALYSIS/data/robbustness_github_gender_swap/r1/'
p_in_g25='/Users/vorsi/Dropbox/behance-data/NEW_ANALYSIS/data/robbustness_github_gender_swap/r25/'


In [14]:
samples=sorted([f for f in os.listdir(p_in_gs) if f.endswith(".csv")])

In [35]:
#open files
github=pd.read_csv('/Users/vorsi/Dropbox/behance-data/NEW_ANALYSIS/data/github/GitHUB_data_sample1_10_01_2021.csv')
joindate=pd.read_csv('/Users/vorsi/Dropbox/behance-data/NEW_ANALYSIS/data/github/users_with_date_creation2505.csv')
joindate_u=pd.read_csv('/Users/vorsi/Dropbox/behance-data/NEW_ANALYSIS/data/github/users_with_date_creation_unknowns.csv')
l_freq=pd.read_csv('/Users/vorsi/Dropbox/behance-data/NEW_ANALYSIS/data/github/users_sample_languages.csv')
u_names=pd.read_csv('/Users/vorsi/Dropbox/behance-data/NEW_ANALYSIS/data/github/asian_names.csv')


In [53]:
#GITHUB
m0_success_ols_github="""
             log_success ~ 
             gender_female +
             tenure +
             n_repos_log +
             reposwhereactive_log
            """


m_success_ols_github="""
             log_success ~  
             gender_female +
             femaleness_2_new_following + 
             gender_female:femaleness_2_new_following +
             tenure +
             n_repos_log +
             reposwhereactive_log
            """

m0_follow_ols_github="""
             followers_log ~  
             gender_female +
             tenure +
             n_repos_log +
             reposwhereactive_log
             """


m_follow_ols_github="""
             followers_log ~ 
               
             gender_female +
             femaleness_2_new_following + 
             gender_female:femaleness_2_new_following +
             tenure +
             n_repos_log +
             reposwhereactive_log
             """


m0_survival_logit_github="""
             still_active ~
             gender_female +
             tenure +
             n_repos_log +
             reposwhereactive_log
             """



m_survival_logit_github ="""
             still_active ~ 
 
             gender_female +
             femaleness_2_new_following + 
             gender_female:femaleness_2_new_following +
             tenure +
             n_repos_log +
             reposwhereactive_log
            """

In [46]:
def get_tenure(row):
    if row['year']!=0:
        return 2016-row['year']
    else:
        return 0

In [79]:
accs=[]

for s in samples:
    df=pd.read_csv(p_in_gs+s)
    df['success']=df['successownrepos'].replace(np.nan,0)
    df['log_success']=np.log10(df["successownrepos"]+1)
    df['followers_log']=np.log10(df['followers']+1)
    df['followf_log']=np.log10(df['followf']+1)
    df['followm_log']=np.log10(df['followm']+1)
    df['followu_log']=np.log10(df['followu']+1)
    df["gendercollaboratorsf_log"]=np.log10(df['gendercollaboratorsf']+1)
    df["gendercollaboratorsm_log"]=np.log10(df['gendercollaboratorsm']+1)
    df["gendercollaboratorsu_log"]=np.log10(df['gendercollaboratorsu']+1)
    df["reposwhereactive_log"]=np.log10(df['reposwhereactive']+1)
    df["pushes_log"]=np.log(df["pushes"]+1)
    df['collaborators_log']=np.log(df['collaborators']+1)
    df['collabs_per_repo']=df['collaborators']/df['reposwhereactive']
    df['pull_request_closed_log']=np.log(df['PRc']+1)
    df['pull_request_opened_log']=np.log(df['PRo']+1)
    df['pull_request_mereged_log']=np.log(df['PRm']+1)
    df['n_repos_log']=np.log(df['n_repos']+1)
    df['tenure']=df.apply(get_tenure,1)
    github_success_m0=smf.ols(formula=m0_success_ols_github, data=df).fit()
    github_success_m=smf.ols(formula=m_success_ols_github, data=df).fit()
    github_follow_m=smf.ols(formula=m_follow_ols_github, data=df).fit()
    github_follow_m0=smf.ols(formula=m0_follow_ols_github, data=df).fit()
    github_survival_m=smf.logit(formula=m_survival_logit_github, data=df).fit()
    github_survival_m0=smf.logit(formula=m0_survival_logit_github, data=df).fit()
    
    #SAVE RESUTLS
    suc_m=save_results(p_out_github_sample, github_success_m, 'github', 'success')
    suc_m0=save_results(p_out_github_sample, github_success_m0, 'github', 'success_0')
    accs.append(suc_m[1])
    accs.append(suc_m0[1])
    
    fol_m=save_results(p_out_github_sample, github_follow_m, 'github', 'follow')
    fol_m0=save_results(p_out_github_sample, github_follow_m0, 'github', 'follow_0')
    accs.append(fol_m[1])
    accs.append(fol_m0[1])
    
    surv_m=save_results(p_out_github_sample, github_survival_m, 'github', 'survival')
    surv_m0=save_results(p_out_github_sample, github_survival_m0, 'github', 'survival_0')
    accs.append(surv_m[1])
    accs.append(surv_m0[1])
    alma

Optimization terminated successfully.
         Current function value: 0.290391
         Iterations 7
Optimization terminated successfully.
         Current function value: 0.291425
         Iterations 7


AttributeError: 'LogitResults' object has no attribute 'rsquared'

In [57]:
p_in1

'/Users/vorsi/Dropbox/behance-data/NEW_ANALYSIS/data/robustness_github/'

In [62]:
p_out_github_sample='/Users/vorsi/Dropbox/behance-data/NEW_ANALYSIS/data/samples_rob_regressions_Github/'

In [66]:
suc_m=save_results(p_out_github_sample, github_success_m, 'github', 'success')
accssuccess.append(suc_m[1])


(                                             Coef.  Std.Err.          t  \
 Intercept                                -0.720013  0.019317 -37.274422   
 gender_female                            -0.000570  0.020017  -0.028453   
 femaleness_2_new_following               -0.141684  0.026462  -5.354235   
 gender_female:femaleness_2_new_following  0.009001  0.035961   0.250301   
 tenure                                    0.085639  0.003008  28.469247   
 n_repos_log                              -0.021661  0.004685  -4.623532   
 reposwhereactive_log                      0.585213  0.014462  40.465987   
 
                                                   P>|t|    [0.025    0.975]  
 Intercept                                 5.054739e-294 -0.757875 -0.682151  
 gender_female                              9.773009e-01 -0.039804  0.038665  
 femaleness_2_new_following                 8.687110e-08 -0.193551 -0.089816  
 gender_female:femaleness_2_new_following   8.023571e-01 -0.061485  0.0794

In [36]:
github_success_m.rsquared

,userlogin,create_date,date_since_reg,year2
0,annuh,2012-10-04,1478,2012
1,sophiecooper,2014-03-13,953,2014
2,nuancesb,2014-10-12,740,2014
3,mobula,2012-08-17,1526,2012
4,SophieMuspratt,2014-03-20,946,2014
...,...,...,...,...
19995,kenkit,2012-05-12,1623,2012
19996,DennisMartinez,2012-01-24,1732,2012
19997,dchacke-ga,2014-11-10,711,2014
19998,c0deous,2013-11-26,1060,2013


In [ ]:
vismodel_nb = smf.glm(formula = formula, 
                                data=df, family=sm.families.NegativeBinomial(alpha=1.0)).fit()

In [50]:
github_success_0=smf.ols(formula=m_success_ols_github, data=df).fit()

In [51]:
#github_success_0.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:            log_success   R-squared:                       0.170
Model:                            OLS   Adj. R-squared:                  0.170
Method:                 Least Squares   F-statistic:                     681.3
Date:                Wed, 15 May 2024   Prob (F-statistic):               0.00
Time:                        10:29:05   Log-Likelihood:                -10389.
No. Observations:               19921   AIC:                         2.079e+04
Df Residuals:                   19914   BIC:                         2.085e+04
Df Model:                           6                                         
Covariance Type:            nonrobust                                         
============================================================================================================
                                               coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------------------------------
Intercept                                   -0.7200      0.019    -37.274      0.000      -0.758      -0.682
gender_female                               -0.0006      0.020     -0.028      0.977      -0.040       0.039
femaleness_2_new_following                  -0.1417      0.026     -5.354      0.000      -0.194      -0.090
gender_female:femaleness_2_new_following     0.0090      0.036      0.250      0.802      -0.061       0.079
tenure                                       0.0856      0.003     28.469      0.000       0.080       0.092
n_repos_log                                 -0.0217      0.005     -4.624      0.000      -0.031      -0.012
reposwhereactive_log                         0.5852      0.014     40.466      0.000       0.557       0.614
==============================================================================
Omnibus:                    12147.857   Durbin-Watson:                   2.007
Prob(Omnibus):                  0.000   Jarque-Bera (JB):           136128.112
Skew:                           2.803   Prob(JB):                         0.00
Kurtosis:                      14.514   Cond. No.                         63.3
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [ ]:
 vismodel_logit05 = smf.logit(formula=formula_top_5,
                                data=df).fit()

In [ ]:
girhub_ = sm.OLS(y, x).fit()
 
# printing the summary table
print(result.summary())

In [26]:
m0_success_ols_B ="""log_project_appr ~ 
             gender_female +
             tenure2 +
             n_projects_log +
             total_activity_log
            """


m_success_ols_B="""
             log_project_appr ~ 
             gender_female +
             femaleness_2_new_following + 
             gender_female:femaleness_2_new_following +
            tenure2 +
             n_projects_log +
             total_activity_log"""


m0_follow_ols_B="""followers_log ~ 
             gender_female +
             tenure2 +
             n_projects_log +
             total_activity_log
            """             

m_follow_ols_B ="""
             followers_log ~ 
             gender_female +
             femaleness_2_new_following + 
             gender_female:femaleness_2_new_following +
             tenure2 +
             n_projects_log +
             total_activity_log
            """

m0_survival_logit_B ="""
             survival ~ 
             gender_female +
             tenure2 +
             n_projects_log +
              total_activity_log
             """

m_survival_logit_B = """
             survival ~ 
             gender_female +
             femaleness_2_new_following + 
             gender_female:femaleness_2_new_following +
             tenure2 +
             n_projects_log +
              total_activity_log"""